In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
batch_size = 32
img_height = 180
img_width = 180


In [ ]:

import pathlib
dataset_path = "/media/regieki/data/RegieKI/data/all_faces2"
data_dir = pathlib.Path(dataset_path)
AUTOTUNE = tf.data.experimental.AUTOTUNE

datasets = {}

for device_dir in data_dir.glob("*/"):
  name = device_dir.stem

  train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    device_dir,
    validation_split=0.1,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

  val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    device_dir,
    validation_split=0.1,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

  datasets[name] = (train_ds, val_ds)

from constants import *


for name, (train_ds, val_ds) in datasets.items():

  print(name)  
  train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
  val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)



## Build the model

Building the neural network requires configuring the layers of the model, then compiling the model.

In [ ]:
from tensorflow import keras

from model.model import getFaceModel

num_classes = len(emotions)

model = getFaceModel()

model.summary()


In [ ]:
checkpoint_path = "models/checkpoints/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

model.load_weights(checkpoint_path)


In [ ]:
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

tb_callback = tf.keras.callbacks.TensorBoard(
    log_dir='logs', histogram_freq=0, write_graph=True, write_images=False
)

epochs=1000

minipoch = 0
for epoch in range(epochs):

  for name, (train_ds, val_ds) in datasets.items():
    
    print(f"Starting {epoch} - {name}")

    history = model.fit(
      train_ds,
      validation_data = val_ds,
      initial_epoch= minipoch,
      epochs= minipoch + 1,
      shuffle=True,
      callbacks=[cp_callback, tb_callback ]
    )
    minipoch += 1





In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()